In [5]:
import pandas as pd

# Load the raw data from an Excel file
data = pd.read_excel(r"C:\Users\naini\Downloads\rawdata.xlsx")

# Display the first few rows to understand the structure
print(data.head())

# Combine 'date' and 'time' columns into a 'datetime' column
data['datetime'] = pd.to_datetime(data['date'].astype(str) + ' ' + data['time'].astype(str))

# Sort data by 'datetime' to ensure proper duration calculation
data = data.sort_values(by='datetime')

# Calculate duration between consecutive events for each sensor (assuming the sensor ID is present)
# We will use the 'shift' method to get the next event's datetime
data['next_datetime'] = data.groupby('sensor')['datetime'].shift(-1)

# Calculate duration in seconds
data['duration'] = (data['next_datetime'] - data['datetime']).dt.total_seconds()

# Replace NaN durations (which occur at the end of each group) with 0 or an appropriate value
data['duration'] = data['duration'].fillna(0)

# Extract the date for grouping
data['date'] = data['datetime'].dt.date

# Group by 'date' and 'location' to calculate total duration for 'inside' and 'outside'
duration_summary = data.groupby(['date', 'location'])['duration'].sum().unstack(fill_value=0).reset_index()

# Filter data for picking and placing activities
picking_placing_data = data[data['activity'].isin(['picking', 'placing'])]

# Group by 'date' and 'activity' to count occurrences
activity_summary = picking_placing_data.groupby(['date', 'activity']).size().unstack(fill_value=0).reset_index()

# Ensure columns for picking and placing exist, even if they have no activities on some dates
activity_summary = activity_summary.reindex(columns=['date', 'picking', 'placing'], fill_value=0)

# Merge duration and activity summaries
result = pd.merge(duration_summary, activity_summary, on='date', how='outer').fillna(0)

# Output the result to an Excel file
result.to_excel('output_summary.xlsx', index=False)

# Display the result
print(result)


        date      time  sensor location  number activity position location.1
0 2024-01-16  10:10:30     0.0       A1       1   placed   inside         A1
1 2024-01-16  10:12:30     1.0       A2       1   picked   inside         A2
2 2024-01-16  10:28:30     1.0       A3       1   picked   Inside         A3
3 2024-01-16  10:38:20     0.0       A4       1   placed   Inside         A4
4 2024-01-16  10:45:20     1.0       A5       1   placed   inside         A5
         date      A1     A10     A11      A12      A13      A14     A15  \
0  2024-01-16  1850.0   840.0  1560.0    540.0    960.0  80830.0   840.0   
1  2024-01-17   660.0   480.0  2040.0  75490.0  75070.0    180.0  2880.0   
2  2024-01-18  4060.0  3340.0   928.0   2100.0    360.0    180.0   780.0   

       A16     A17  ...    A33  A34      A4      A5      A6      A7      A8  \
0  80230.0   540.0  ...    0.0  0.0  1020.0  2100.0  2220.0   900.0  1200.0   
1      0.0     0.0  ...    0.0  0.0  4260.0   240.0   600.0    60.0   960.0